In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus

C:\Users\argenisleon\Anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable

    You are using PySparkling of version 2.4.10, but your PySpark is of
    version 2.3.1. Please make sure Spark and PySparkling versions are compatible. 


In [4]:
op= Optimus(master="local", app_name= "optimus", verbose = True)

5Jrn-biCd06mFKFUgrLMcxSWnzv8EbsKO9zpgWbcSOY=
b772aa34-09e2-4eb8-a57e-b147356a96b7


INFO:optimus:Operative System:Windows
INFO:optimus:Just check that Spark and all necessary environments vars are present...
INFO:optimus:-----
INFO:optimus:SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
INFO:optimus:HADOOP_HOME=C:\opt\hadoop-2.7.7
INFO:optimus:PYSPARK_PYTHON=C:\Users\argenisleon\Anaconda3\python.exe
INFO:optimus:PYSPARK_DRIVER_PYTHON=jupyter
INFO:optimus:PYSPARK_SUBMIT_ARGS=--jars "file:///C:/Users/argenisleon/Documents/Optimus/optimus/jars/RedshiftJDBC42-1.2.16.1027.jar,file:///C:/Users/argenisleon/Documents/Optimus/optimus/jars/mysql-connector-java-8.0.16.jar,file:///C:/Users/argenisleon/Documents/Optimus/optimus/jars/ojdbc8.jar,file:///C:/Users/argenisleon/Documents/Optimus/optimus/jars/postgresql-42.2.5.jar" --driver-class-path "C:/Users/argenisleon/Documents/Optimus/optimus/jars/RedshiftJDBC42-1.2.16.1027.jar;C:/Users/argenisleon/Documents/Optimus/optimus/jars/mysql-connector-java-8.0.16.jar;C:/Users/argenisleon/Documents/Optimus/optimus/jars/ojdbc8.jar;C:/User

In [5]:
import pandas as pd
from pyspark.sql.types import *
from datetime import date, datetime
df = op.load.csv("data/Meteorite_Landings.csv").h_repartition()

In [6]:
from optimus.bumblebee import Comm

In [7]:
df.send()

Including 'nan' as Null in processing string type column'name'
Including 'nan' as Null in processing string type column'nametype'
Including 'nan' as Null in processing string type column'recclass'
Including 'nan' as Null in processing string type column'fall'
Including 'nan' as Null in processing string type column'year'
Including 'nan' as Null in processing string type column'GeoLocation'


INFO:optimus:agg_exprs() executed in 29.68 sec
INFO:optimus:general_stats() executed in 29.68 sec
INFO:optimus:Processing column count na'name'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'id'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'nametype'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'recclass'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'mass (g)'...
INFO:optimus:_count_data_types() executed in 0.01 sec
INFO:optimus:Processing column count na'fall'...
INFO:optimus:_count_data_types() executed in 0.01 sec
INFO:optimus:Processing column count na'year'...
INFO:optimus:_count_data_types() executed in 0.01 sec
INFO:optimus:Processing column count na'reclat'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'reclong'...
INFO:optimus:_count

gAAAAABdS0JhrOb6u0Z_FLU3WusUYdSxNm2Fj9CMAJ6jzB7BfE40lnMus3WLhWx9F1zJd6k65aE3-S5qTxTcHwIMVo6EmhUxZEEwlpxI5MoRZ8mSEHtZQh3l7xQmxOhtbEot0GSmyebPnzZh88mc7S1KvA4_VPdYc0IRrYxwz_Igkx3qEs4psHcf7xlKCx8sEB4gP_MMZ64QOhcQPpcvp13ZzNZ452TbNyZRO3XixGqckSrDvMJ15fDI1os1aOXcLu4fUGWKHqtrkSHX4EYjsAAXXp6VozjyDhqhBbt2Qk7R_3XM0L_w3iwRGob-TabcO8WF2tPAWbdR20oe-agqRtAbufTBM9GhIBeGupSAuFVeE0iNRMasRfSi4iWRfw_2ywprXm7pec2tizoBfIx2hu_1alt4zl_JnLoXGtxzSdQEWR9Qf5C5bSFA9X3ckQPIr-JtIJxxGIVYg5XhJJqa7k_9HtNhnAtw48baJ8mARcFAw3qxCoiITGkgtX_5EvEJQgKxgc-gCCq8Hkg9uqoldugWX6X5e_kwPG3sTyQUtZGDGNdmp0RkVx_Egws5sBqmR8NS47xYlHr-kHH5Es-OW3IkF1GAW28JFQmwoRa4CBHRy12oOA9M3xhZgAGCJka61NO6Dmvq_Or22vUewQRYHwCjpC2C1EryhYeVarX9FcCi_UxtjlhOdP9ILFtxrMSWEkWfh6D9vDSFeBEYnCxMKid56gJBiLYwRcjPiW_QuW8uPt5H4U12Q8rYlUc9K031GxNHhNLtorMlr1jpV0DEgSZLsjESgpiojW9dFAu17IV-5Cw5k1aAj3DibrCMmUi1GXRV_74Ns2EbO_z_za2pM2ljstHgr8HqP4U-uyR6goRFyRl1CJBgMkAksp66UMN6Duyyp5VB5o6qESoUu3xEexyiwRPibICo-0KAw2dYWKCPmE70_Lw4ieSDTsko7hjAh1Ydh9XcZojI643ljdWtQLbZfd3rv94QUJvrYUdf

In [46]:
df.select(["name", "id","recclass"]).send()

AttributeError: type object 'Profiler' has no attribute 'instance'

In [11]:
df.cols.dtypes()

{'name': 'string',
 'id': 'int',
 'nametype': 'string',
 'recclass': 'string',
 'mass (g)': 'double',
 'fall': 'string',
 'year': 'string',
 'reclat': 'double',
 'reclong': 'double',
 'GeoLocation': 'string'}

In [74]:
c= Comm()

5Jrn-biCd06mFKFUgrLMcxSWnzv8EbsKO9zpgWbcSOY=
b772aa34-09e2-4eb8-a57e-b147356a96b7


In [75]:
c.send("hola1")

Data sent 



In [110]:
df.cols.dtypes()

{'name': 'string',
 'id': 'int',
 'nametype': 'string',
 'recclass': 'string',
 'mass (g)': 'double',
 'fall': 'string',
 'year': 'string',
 'reclat': 'double',
 'reclong': 'double',
 'GeoLocation': 'string'}

In [61]:
[{"title":c} for c in df.columns]

[{'title': 'name'},
 {'title': 'id'},
 {'title': 'nametype'},
 {'title': 'recclass'},
 {'title': 'mass (g)'},
 {'title': 'fall'},
 {'title': 'year'},
 {'title': 'reclat'},
 {'title': 'reclong'},
 {'title': 'GeoLocation'}]

In [9]:
from optimus.profiler.profiler import Profiler
p = Profiler()
p.to_json(df, "*")

INFO:optimus:Config.ini not found


Including 'nan' as Null in processing string type column'name'
Including 'nan' as Null in processing string type column'nametype'
Including 'nan' as Null in processing string type column'recclass'
Including 'nan' as Null in processing string type column'fall'
Including 'nan' as Null in processing string type column'year'
Including 'nan' as Null in processing string type column'GeoLocation'


INFO:optimus:agg_exprs() executed in 22.16 sec
INFO:optimus:general_stats() executed in 22.16 sec
INFO:optimus:Processing column count na'name'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'id'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'nametype'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'recclass'...
INFO:optimus:_count_data_types() executed in 0.01 sec
INFO:optimus:Processing column count na'mass (g)'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'fall'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'year'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'reclat'...
INFO:optimus:_count_data_types() executed in 0.0 sec
INFO:optimus:Processing column count na'reclong'...
INFO:optimus:_count_d

'{"columns": {"name": {"stats": {"missing_count": 0, "p_missing": 0.0, "stats": {}}, "frequency": [{"value": "Yamato 791240", "count": 1, "percentage": 10.0}, {"value": "Northwest Africa 5064", "count": 1, "percentage": 10.0}, {"value": "Miller Butte 03001", "count": 1, "percentage": 10.0}, {"value": "Lewis Cliff 86126", "count": 1, "percentage": 10.0}, {"value": "LaPaz Icefield 04834", "count": 1, "percentage": 10.0}, {"value": "Grove Mountains 052321", "count": 1, "percentage": 10.0}, {"value": "Grove Mountains 052260", "count": 1, "percentage": 10.0}, {"value": "Grove Mountains 020129", "count": 1, "percentage": 10.0}, {"value": "Geologists Range 99101", "count": 1, "percentage": 10.0}, {"value": "Elephant Moraine 90033", "count": 1, "percentage": 10.0}], "column_dtype": "string", "name": "name", "column_type": "categorical", "dtypes_stats": {"string": 10, "bool": 0, "int": 0, "float": 0, "double": 0, "date": 0, "array": 0, "null": 0, "missing": 0}}, "id": {"stats": {"min": 8443, "m

In [45]:
from pyspark.sql import functions as F
from itertools import product

def a(funcs, columns):
    return [(x,y) for x,y in product(funcs, columns)]
        
a([F.min, F.max, F.mean], ["name","id"])

[(<function pyspark.sql.functions._create_function.<locals>._(col)>, 'name'),
 (<function pyspark.sql.functions._create_function.<locals>._(col)>, 'id'),
 (<function pyspark.sql.functions._create_function.<locals>._(col)>, 'name'),
 (<function pyspark.sql.functions._create_function.<locals>._(col)>, 'id'),
 (<function pyspark.sql.functions._create_function.<locals>._(col)>, 'name'),
 (<function pyspark.sql.functions._create_function.<locals>._(col)>, 'id')]

In [8]:
df.select("id").send()

AttributeError: type object 'Profiler' has no attribute 'instance'

In [129]:
def a(ar):
    self.hola = 1
a.hola

AttributeError: 'function' object has no attribute 'hola'

In [121]:
df.cols.count_na("id")

{'name': 0,
 'id': 0,
 'nametype': 0,
 'recclass': 0,
 'mass (g)': 0,
 'fall': 0,
 'year': 0,
 'reclat': 0,
 'reclong': 0,
 'GeoLocation': 0}

In [7]:
df.table()

name 1 (string) nullable,id 2 (int) nullable,nametype 3 (string) nullable,recclass 4 (string) nullable,mass (g) 5 (double) nullable,fall 6 (string) nullable,year 7 (string) nullable,reclat 8 (double) nullable,reclong 9 (double) nullable,GeoLocation 10 (string) nullable
Dominion⋅Range⋅08357,52132,Valid,L6,8.9,Found,01/01/2008⋅12:00:00⋅AM,0.0,0.0,"(0.000000,⋅0.000000)"
Yamato⋅792863,28212,Valid,H5,132.25,Found,01/01/1979⋅12:00:00⋅AM,-71.5,35.66667,"(-71.500000,⋅35.666670)"
Acfer⋅232,240,Valid,H5,725.0,Found,01/01/1991⋅12:00:00⋅AM,27.73944,4.32833,"(27.739440,⋅4.328330)"
Jiddat⋅al⋅Harasis⋅755,56470,Valid,L5,17.5,Found,None,19.83528,56.46139,"(19.835280,⋅56.461390)"
Imlay,52855,Valid,L5,770.0,Found,01/01/2009⋅12:00:00⋅AM,40.74018,-118.17285,"(40.740180,⋅-118.172850)"
Shişr⋅029,23564,Valid,H5,204.71,Found,01/01/1998⋅12:00:00⋅AM,18.1501,53.80047,"(18.150100,⋅53.800470)"
Northwest⋅Africa⋅3148,31272,Valid,Eucrite,138.0,Found,01/01/2004⋅12:00:00⋅AM,None,None,None
Dhofar⋅1462,55271,Valid,L~4,400.8,Found,01/01/2008⋅12:00:00⋅AM,18.34955,54.25013,"(18.349550,⋅54.250130)"
Dhofar⋅085,6784,Valid,H4,78.0,Found,01/01/1999⋅12:00:00⋅AM,19.21233,54.8475,"(19.212330,⋅54.847500)"
Grove⋅Mountains⋅053621,50563,Valid,L6,1.8,Found,01/01/2006⋅12:00:00⋅AM,-72.78194,75.30056,"(-72.781940,⋅75.300560)"


In [10]:
%%time
df.cols.frequency("fall")

Wall time: 2.78 s


{'fall': [{'value': 'Found', 'count': 44609},
  {'value': 'Fell', 'count': 1107}]}

In [13]:
%%time
df.table_name("aaa")
query = []
columns = ["fall"]
for c in columns:
    query.append("`{COL_NAME}`, count(*) over (partition BY `{COL_NAME}`) as `c_{COL_NAME}`".format(COL_NAME=c))
query = "SELECT " + ",".join(query) + " FROM aaa"
print(query)

p= df.query(query)
p.show()


SELECT `fall`, count(*) over (partition BY `fall`) as `c_fall` FROM aaa
+----+------+
|fall|c_fall|
+----+------+
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
|Fell|  1107|
+----+------+
only showing top 20 rows

Wall time: 1.16 s


In [235]:
%%time
p.table()

name 1 (string) nullable,c_name 2 (bigint) not nullable,id 3 (int) nullable,c_id 4 (bigint) not nullable,nametype 5 (string) nullable,c_nametype 6 (bigint) not nullable,recclass 7 (string) nullable,c_recclass 8 (bigint) not nullable,mass (g) 9 (double) nullable,c_mass (g) 10 (bigint) not nullable,fall 11 (string) nullable,c_fall 12 (bigint) not nullable,year 13 (string) nullable,c_year 14 (bigint) not nullable,reclat 15 (double) nullable,c_reclat 16 (bigint) not nullable,reclong 17 (double) nullable,c_reclong 18 (bigint) not nullable,GeoLocation 19 (string) nullable,c_GeoLocation 20 (bigint) not nullable
Grove⋅Mountains⋅022595,1,47999,1,Valid,45641,H4,4211,0.66,19,Found,44609,01/01/2003⋅12:00:00⋅AM,3323,-72.7825,39,75.279167,1,"(-72.782500,⋅75.279167)",1
Yamato⋅86590,1,30096,1,Valid,45641,H4,4211,0.66,19,Found,44609,01/01/1986⋅12:00:00⋅AM,1375,-71.5,4761,35.66667,4985,"(-71.500000,⋅35.666670)",4761
Grove⋅Mountains⋅022198,1,47919,1,Valid,45641,L6,8285,0.66,19,Found,44609,01/01/2003⋅12:00:00⋅AM,3323,-72.781389,8,75.305556,3,"(-72.781389,⋅75.305556)",2
Grove⋅Mountains⋅022758,1,48015,1,Valid,45641,H6,4528,0.66,19,Found,44609,01/01/2003⋅12:00:00⋅AM,3323,-72.777778,11,75.279444,4,"(-72.777778,⋅75.279444)",1
Grove⋅Mountains⋅021769,1,49878,1,Valid,45641,CR2,135,0.66,19,Found,44609,01/01/2003⋅12:00:00⋅AM,3323,-72.7844,1,75.3094,1,"(-72.784400,⋅75.309400)",1
Frontier⋅Mountain⋅90199,1,10589,1,Valid,45641,H4,4211,0.66,19,Found,44609,01/01/1990⋅12:00:00⋅AM,1518,-72.95591,1,160.54133,1,"(-72.955910,⋅160.541330)",1
Grove⋅Mountains⋅023784,1,48118,1,Valid,45641,L5,4796,0.66,19,Found,44609,01/01/2003⋅12:00:00⋅AM,3323,-72.984722,4,75.249444,5,"(-72.984722,⋅75.249444)",1
Yamato⋅8413,1,29460,1,Valid,45641,Eucrite,221,0.66,19,Found,44609,01/01/1984⋅12:00:00⋅AM,402,-71.5,4761,35.66667,4985,"(-71.500000,⋅35.666670)",4761
Grove⋅Mountains⋅022793,1,48020,1,Valid,45641,LL6,2043,0.66,19,Found,44609,01/01/2003⋅12:00:00⋅AM,3323,-72.775556,11,75.338611,7,"(-72.775556,⋅75.338611)",2
Yamato⋅86557,1,30063,1,Valid,45641,H4,4211,0.66,19,Found,44609,01/01/1986⋅12:00:00⋅AM,1375,-71.5,4761,35.66667,4985,"(-71.500000,⋅35.666670)",4761


Wall time: 14min 29s


In [244]:
import numpy as np
print(pdf)
count,division = np.histogram(pdf['function'])

           names  c_names  height(ft)  c_height(ft)          function  \
0              1        1         2.0             1                 3   
1  Metroplex_)^$        1       300.0             1    Battle Station   
2   bumbl#ebéé          1        17.0             1         Espionage   
3       Optim'us        1        28.0             1            Leader   
4       Megatron        1         NaN             1              None   
5           Jazz        1        13.0             1  First Lieutenant   
6      ironhide&        1        26.0             1          Security   

   c_function  rank  c_rank      age  c_age  ...    DateType  c_DateType  \
0           1     4       1        5      1  ...  2011-04-10           2   
1           1     8       2  5000000      6  ...  2011-04-10           2   
2           1     7       2  5000000      6  ...  2015-08-10           1   
3           1    10       2  5000000      6  ...  2016-09-10           1   
4           1    10       2  500000

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [232]:
#Process columns one by one

%%time
df.cols.frequency("name",40)
df.cols.frequency("id",40)
df.cols.frequency("nametype",40)
df.cols.frequency("recclass",40)
df.cols.frequency("mass (g)",40)
df.cols.frequency("fall",40)
df.cols.frequency("year",40)
df.cols.frequency("reclat",40)
df.cols.frequency("reclong",40)
df.cols.frequency("GeoLocation",40)



Wall time: 1min 27s


In [233]:
# Process all the columns
%%time
df.cols.frequency('*',40)

Wall time: 1min 28s


{'name': [{'value': 'Święcany', 'count': 1},
  {'value': 'Łowicz', 'count': 1},
  {'value': 'Österplana 064', 'count': 1},
  {'value': 'Österplana 063', 'count': 1},
  {'value': 'Österplana 062', 'count': 1},
  {'value': 'Österplana 061', 'count': 1},
  {'value': 'Österplana 060', 'count': 1},
  {'value': 'Österplana 059', 'count': 1},
  {'value': 'Österplana 058', 'count': 1},
  {'value': 'Österplana 057', 'count': 1},
  {'value': 'Österplana 056', 'count': 1},
  {'value': 'Österplana 055', 'count': 1},
  {'value': 'Österplana 054', 'count': 1},
  {'value': 'Österplana 053', 'count': 1},
  {'value': 'Österplana 052', 'count': 1},
  {'value': 'Österplana 051', 'count': 1},
  {'value': 'Österplana 050', 'count': 1},
  {'value': 'Österplana 049', 'count': 1},
  {'value': 'Österplana 048', 'count': 1},
  {'value': 'Österplana 047', 'count': 1},
  {'value': 'Österplana 046', 'count': 1},
  {'value': 'Österplana 045', 'count': 1},
  {'value': 'Österplana 044', 'count': 1},
  {'value': 'Öste

In [220]:
%%time
# p= df.query("SELECT * FROM (SELECT `fall`, count(*) over (partition BY `fall`) as `c_fall` FROM aaa) ORDER BY `c_fall` DESC LIMIT 10")
p= df.query("SELECT `fall`, count(*) over (partition BY `fall`) as `c_fall` FROM aaa")

p.table()

fall 1 (string) nullable,c_fall 2 (bigint) not nullable
Fell,1107
Fell,1107
Fell,1107
Fell,1107
Fell,1107
Fell,1107
Fell,1107
Fell,1107
Fell,1107
Fell,1107


Wall time: 1.91 s


In [170]:
p.table()

names 1 (string) nullable,c_names 2 (bigint) not nullable,height(ft) 3 (int) nullable,c_height(ft) 4 (bigint) not nullable,function 5 (string) nullable,c_function 6 (bigint) not nullable,rank 7 (int) nullable,c_rank 8 (bigint) not nullable,age 9 (int) nullable,c_age 10 (bigint) not nullable,weight(t) 11 (float) nullable,c_weight(t) 12 (bigint) not nullable,japanese name 13 (array<string>) nullable,c_japanese name 14 (bigint) not nullable,last position seen 15 (string) nullable,c_last position seen 16 (bigint) not nullable,date arrival 17 (string) nullable,c_date arrival 18 (bigint) not nullable,last date seen 19 (string) nullable,c_last date seen 20 (bigint) not nullable,attributes 21 (array<float>) nullable,c_attributes 22 (bigint) not nullable,DateType 23 (date) nullable,c_DateType 24 (bigint) not nullable,Timestamp 25 (timestamp) nullable,c_Timestamp 26 (bigint) not nullable,Cybertronian 27 (boolean) nullable,c_Cybertronian 28 (bigint) not nullable,function(binary) 29 (binary) not nullable,c_function(binary) 30 (bigint) not nullable,NullType 31 (null) nullable,c_NullType 32 (bigint) not nullable
1,1,2,1,3,1,4,1,5,1,6.0,1,['7'],1,8,1,1980/04/10,7,2011/04/10,2,[11.0],1,2011-04-10,2,2014-06-24⋅00:00:00,7,True,7,bytearray(b'15'),1,None,7
Metroplex_)^$,1,300,1,Battle⋅Station,1,8,2,5000000,6,None,1,['Metroflex'],1,None,2,1980/04/10,7,2011/04/10,2,"[91.44000244140625,⋅None]",1,2011-04-10,2,2014-06-24⋅00:00:00,7,True,7,bytearray(b'Battle⋅Station'),1,None,7
bumbl#ebéé⋅⋅,1,17,1,Espionage,1,7,2,5000000,6,2.0,1,"['Bumble',⋅'Goldback']",1,"10.642707,-71.612534",1,1980/04/10,7,2015/08/10,1,"[5.334000110626221,⋅2000.0]",1,2015-08-10,1,2014-06-24⋅00:00:00,7,True,7,bytearray(b'Espionage'),1,None,7
Optim'us,1,28,1,Leader,1,10,2,5000000,6,4.300000190734863,1,"['Inochi',⋅'Convoy']",1,"19.442735,-99.201111",1,1980/04/10,7,2016/09/10,1,"[8.53439998626709,⋅4300.0]",1,2016-09-10,1,2014-06-24⋅00:00:00,7,True,7,bytearray(b'Leader'),1,None,7
Megatron,1,None,1,None,1,10,2,5000000,6,5.699999809265137,1,['Megatron'],1,None,2,1980/04/10,7,2012/05/10,1,"[None,⋅5700.0]",1,2012-05-10,1,2014-06-24⋅00:00:00,7,True,7,bytearray(b'None'),1,None,7
Jazz,1,13,1,First⋅Lieutenant,1,8,2,5000000,6,1.7999999523162842,1,['Meister'],1,"33.670666,-117.841553",1,1980/04/10,7,2013/06/10,1,"[3.962399959564209,⋅1800.0]",1,2013-06-24,1,2014-06-24⋅00:00:00,7,True,7,bytearray(b'First⋅Lieutenant'),1,None,7
ironhide&,1,26,1,Security,1,7,2,5000000,6,4.0,1,['Roadbuster'],1,"37.789563,-122.400356",1,1980/04/10,7,2014/07/10,1,"[7.924799919128418,⋅4000.0]",1,2014-06-24,1,2014-06-24⋅00:00:00,7,True,7,bytearray(b'Security'),1,None,7


In [169]:
pdf = p.toPandas()

In [161]:
# print(pdf)
# pdf=pdf.groupby(['names','c_names']).size()
for c in columns:    
    temp = pdf[[c,"c_"+c]].drop_duplicates()
    for index,row in temp.iterrows():
        print(row["names"], row["c_names"])

# for index, row in pdf.iterrows():
#     print(pdf["names"], pdf["c_names"])

1 1
Metroplex_)^$ 1
bumbl#ebéé   1
Optim'us 1
Megatron 1
Jazz 1
ironhide& 1


KeyError: 'names'

In [98]:
%%time
df.cols.frequency("*")

Wall time: 43 s


{'names': [{'value': 'ironhide&', 'count': 1},
  {'value': 'bumbl#ebéé  ', 'count': 1},
  {'value': "Optim'us", 'count': 1},
  {'value': 'Metroplex_)^$', 'count': 1},
  {'value': 'Megatron', 'count': 1},
  {'value': 'Jazz', 'count': 1},
  {'value': '1', 'count': 1}],
 'height(ft)': [{'value': 300, 'count': 1},
  {'value': 28, 'count': 1},
  {'value': 26, 'count': 1},
  {'value': 17, 'count': 1},
  {'value': 13, 'count': 1},
  {'value': 2, 'count': 1},
  {'value': None, 'count': 1}],
 'function': [{'value': 'Security', 'count': 1},
  {'value': 'None', 'count': 1},
  {'value': 'Leader', 'count': 1},
  {'value': 'First Lieutenant', 'count': 1},
  {'value': 'Espionage', 'count': 1},
  {'value': 'Battle Station', 'count': 1},
  {'value': '3', 'count': 1}],
 'rank': [{'value': 10, 'count': 2},
  {'value': 8, 'count': 2},
  {'value': 7, 'count': 2},
  {'value': 4, 'count': 1}],
 'age': [{'value': 5000000, 'count': 6}, {'value': 5, 'count': 1}],
 'weight(t)': [{'value': 6.0, 'count': 1},
  {'v

In [8]:
df.select("names").send()

INFO:optimus:Processing column 'names'...


Including 'nan' as Null in processing 'names'


INFO:optimus:_count_data_types() executed in 2.24 sec
INFO:optimus:count_data_types() executed in 2.25 sec
INFO:optimus:Using 'column_exp' to process column 'names' with function _cast_to
INFO:optimus:cast_columns() executed in 0.02 sec


'stddev' function in 'names' column is returning 'nan'. Is that what you expected?. Seems that 'names' has 'nan' values
'kurtosis' function in 'names' column is returning 'nan'. Is that what you expected?. Seems that 'names' has 'nan' values
'skewness' function in 'names' column is returning 'nan'. Is that what you expected?. Seems that 'names' has 'nan' values
'variance' function in 'names' column is returning 'nan'. Is that what you expected?. Seems that 'names' has 'nan' values


INFO:optimus:agg_exprs() executed in 1.15 sec
INFO:optimus:general_stats() executed in 1.16 sec
INFO:optimus:------------------------------
INFO:optimus:Processing column 'names'...
INFO:optimus:frequency() executed in 1.28 sec
INFO:optimus:stats_by_column() executed in 0.0 sec
INFO:optimus:Using 'column_exp' to process column 'names_len' with function func_col_exp
INFO:optimus:Using 'column_exp' to process column 'names_len' with function _bucketizer
INFO:optimus:hist() executed in 1.49 sec
INFO:optimus:hist_string() executed in 3.66 sec


Including 'nan' as Null in processing 'names'


INFO:optimus:dataset_info() executed in 2.15 sec
INFO:optimus:Send to the queue


In [4]:
from optimus.bumblebee import Comm

C:\Users\argenisleon\Anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable

    You are using PySparkling of version 2.4.10, but your PySpark is of
    version 2.3.1. Please make sure Spark and PySparkling versions are compatible. 


In [22]:
c = Comm()
c.send({"a":1})

queue_exchange 
queue_routing_key f657036e-90b5-4391-8d48-fb3f16f6b8ae


In [10]:
new_df = df\
    .cols.lower(["names","function"])\
    .cols.date_transform("date arrival", "yyyy/MM/dd", "dd-MM-YYYY")\
    .cols.years_between("date arrival", "dd-MM-YYYY", output_cols = "from arrival")\
    .cols.remove_accents("names")\
    .cols.remove_special_chars("names")\
    .rows.drop(df["rank"]>8)\
    .cols.rename(str.lower)\
    .cols.trim("*")\
    .cols.unnest("japanese name", output_cols="other names")\
    .cols.unnest("last position seen",separator=",", output_cols="pos")\
    .cols.drop(["last position seen", "japanese name","date arrival", "cybertronian", "nulltype"])

INFO:optimus:Using 'column_exp' to process column 'names' with function _lower
INFO:optimus:Using 'column_exp' to process column 'function' with function _lower
INFO:optimus:Using 'column_exp' to process column 'date arrival' with function _date_transform
INFO:optimus:Using 'column_exp' to process column 'date arrival' with function _years_between
INFO:optimus:Using 'column_exp' to process column 'from arrival' with function _cast_to
INFO:optimus:Using 'pandas_udf' to process column 'names' with function _remove_accents
INFO:optimus:Using 'pandas_udf' to process column 'names' with function multiple_replace
INFO:optimus:`japanese name`,`attributes`,`timestamp`,`function(binary)`,`nulltype` column(s) was not processed because is/are not byte,short,big,int,double,float,string,date,bool
INFO:optimus:Using 'column_exp' to process column 'names' with function _trim
INFO:optimus:Using 'column_exp' to process column 'height(ft)' with function _trim
INFO:optimus:Using 'column_exp' to process c

In [ ]:
df.select("names").send()

In [29]:
bb.df("transformers").table()

names 1 (string) nullable,height(ft) 2 (int) nullable,function 3 (string) nullable,rank 4 (int) nullable,age 5 (int) nullable,weight(t) 6 (float) nullable,japanese name 7 (array<string>) nullable,last position seen 8 (string) nullable,date arrival 9 (string) nullable,last date seen 10 (string) nullable,attributes 11 (array<float>) nullable,DateType 12 (date) nullable,Tiemstamp 13 (timestamp) nullable,Cybertronian 14 (boolean) nullable,function(binary) 15 (binary) not nullable,NullType 16 (null) nullable
Optim'us,28,Leader,10,5000000,4.300000190734863,"['Inochi',⋅'Convoy']","19.442735,-99.201111",1980/04/10,2016/09/10,"[8.53439998626709,⋅4300.0]",2016-09-10,2014-06-24⋅00:00:00,True,bytearray(b'Leader'),None
bumbl#ebéé⋅⋅,17,Espionage,7,5000000,2.0,"['Bumble',⋅'Goldback']","10.642707,-71.612534",1980/04/10,2015/08/10,"[5.334000110626221,⋅2000.0]",2015-08-10,2014-06-24⋅00:00:00,True,bytearray(b'Espionage'),None
ironhide&,26,Security,7,5000000,4.0,['Roadbuster'],"37.789563,-122.400356",1980/04/10,2014/07/10,"[7.924799919128418,⋅4000.0]",2014-06-24,2014-06-24⋅00:00:00,True,bytearray(b'Security'),None
Jazz,13,First⋅Lieutenant,8,5000000,1.7999999523162842,['Meister'],"33.670666,-117.841553",1980/04/10,2013/06/10,"[3.962399959564209,⋅1800.0]",2013-06-24,2014-06-24⋅00:00:00,True,bytearray(b'First⋅Lieutenant'),None
Megatron,None,None,10,5000000,5.699999809265137,['Megatron'],None,1980/04/10,2012/05/10,"[None,⋅5700.0]",2012-05-10,2014-06-24⋅00:00:00,True,bytearray(b'None'),None
Metroplex_)^$,300,Battle⋅Station,8,5000000,None,['Metroflex'],None,1980/04/10,2011/04/10,"[91.44000244140625,⋅None]",2011-04-10,2014-06-24⋅00:00:00,True,bytearray(b'Battle⋅Station'),None
1,2,3,4,5,6.0,['7'],8,1980/04/10,2011/04/10,[11.0],2011-04-10,2014-06-24⋅00:00:00,True,bytearray(b'15'),None


In [10]:
bb.dfs

{'transformers': DataFrame[names: string, height(ft): int, function: string, rank: int, age: int, weight(t): float, japanese name: array<string>, last position seen: string, date arrival: string, last date seen: string, attributes: array<float>, DateType: date, Tiemstamp: timestamp, Cybertronian: boolean, function(binary): binary, NullType: null]}

In [17]:
bb.drop("transformers")

'transformers' dataframe do not exist


In [246]:
import pandas as pd

from pyspark.sql.functions import col, pandas_udf, struct, PandasUDFType

pdf = pd.DataFrame([1, 2, 3], columns=["x"])
df = op.spark.createDataFrame(pdf)

# When the UDF is called with a single column that is not StructType,
# the input to the underlying function is an iterator of pd.Series.
@pandas_udf("long", PandasUDFType.SCALAR_ITER)
def plus_one(batch_iter):
    for x in batch_iter:
        yield x + 1

df.select(plus_one(col("x"))).show()


AttributeError: type object 'PandasUDFType' has no attribute 'SCALAR_ITER'

In [249]:
df.table()

x 1 (bigint) nullable
1
2
3


In [ ]:
df.cols.stratify("name")

In [ ]:
l
